2019/09/28 由曾元顯建立，
2020/08/24修改

#壹、設置並查詢 GPU
在 Colab 畫面中，點擊 編輯 > 筆記本設定 
或者 
執行階段 > 變更執行階段類型 ，選擇 GPU 做為 硬體加速器

In [ ]:
# 查詢 GPU
!nvidia-smi

Mon Aug 24 09:23:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

底下參考：https://leowood.github.io/2018/11/16/BERT-fine-tuning-with-Google-GPU/

#貳、設定 Colab 權限以連結本機上的雲端硬碟 
運行下面的程式碼，會出現鏈接，點開鏈接，選擇自己的谷歌帳號，將得到的代碼（類似：「4/rgFOTdBFqoSFMoinr_9uzzTLziWUyGjSQgNLt1EV4gbrM6gBYp-Vnxk」），拷貝後輸入鏈接下面的框中即可。

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


設置工作路徑：

In [ ]:
import os, time
os.chdir( '/content/gdrive/My Drive' ) # change directory to your GoogleDrive

In [ ]:
!ls BERT/Chinese_Skewed_TxtClf/BERT_txtclf

將筆記本的工作路徑設置到BERT代碼的文件夾下。

可以用ls命令查看是否成功：

In [ ]:
os.chdir('BERT/Chinese_Skewed_TxtClf/BERT_txtclf') # change directory to desired working directory
!ls .

eval_clf.py		    run_classifier_sam.py
run_classifier_colab.ipynb  train_test_split.py


#參、安裝舊版的 tesorflow 以便執行 Google 釋出的程式

由於現在連上 Colab，都使用新版 tensorflow，要先解除。

再安裝舊版的 tensorflow，以搭配 Google 釋出的程式：https://github.com/google-research/bert

In [ ]:
!pip uninstall tensorflow
# 會被問：Proceed (y/n)? 記得要按「y]

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.3.0


In [ ]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 512kB 35.8MB/s 
     |████████████████████████████████| 3.8MB 35.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=e05c9b5e8fbd30c5ec8330b98befa94a3db5b026f6000de45458bf5995dfd39d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninsta

#肆、訓練、預測、評估
##一、CnonC 資料集
### 1. 訓練 CnonC
在 colab.research.google.com 上執行，大約7分鐘。
###注意1：下面的訓練命令，若執行有誤，需刪除 Model_CnonC 再重新執行：
在本機端以滑鼠將 Model_CnonC 目錄丟到垃圾桶，並等到 GoogleDrive 同步完成，再重新執行訓練程式！

或是新增「代碼」格子，輸入並執行：!rm -rf Model_CnonC

但以本機端刪除Model_CnonC並等待同步完成為佳！
其他的資料集，若訓練過程有誤，需重新訓練，也要刪除其 model 目錄的內容，再重新執行，訓練後的權重資料可能會比較乾淨，評估成效時，比較不會有問題。

###注意2：下面的訓練命令，注意其路徑的正確性。

###注意3：當前的目錄下，至少要有這幾個 Google 釋出的檔案：

extract_features.py

modeling.py

optimization.py

tokenization.py


In [ ]:
!ls

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=cnonc --do_train=true --do_eval=false \
  --data_dir=../Datasets \
  --vocab_file=bert/chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=bert/chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=bert/chinese_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=16 \
  --learning_rate=2e-5 \
  --num_train_epochs=20.0 \
  --output_dir=Model_CnonC
print("It takes %4.2f seconds to train CnonC."%(time.time()-time_Start))



W0824 10:03:59.699859 139681055766400 module_wrapper.py:139] From /content/gdrive/My Drive/BERT/Chinese_Skewed_TxtClf/BERT_txtclf/modeling.py:91: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0824 10:04:01.767512 139681055766400 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file

若出現其他錯誤，可以執行下面這一行，斷開連結，重新連接虛擬機(主要使用時機為清除佔據記憶體的變數)：

In [ ]:
#!kill -9 -1

###2、預測 CnonC

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=cnonc --do_predict=true \
  --data_dir=../Datasets \
  --vocab_file=bert/chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=bert/chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=Model_CnonC \
  --max_seq_length=128 \
  --output_dir=Output_CnonC
print("It takes %4.2f seconds to predict CnonC."%(time.time()-time_Start))



W0824 10:15:49.720468 140312741132160 module_wrapper.py:139] From /content/gdrive/My Drive/BERT/Chinese_Skewed_TxtClf/BERT_txtclf/modeling.py:91: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0824 10:15:49.791982 140312741132160 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file

###3.評估 CnonC 成效
eval_clf.py 裡面的資料集名稱，必須要跟第一個參數對應，例如下面的：CnonC，則 eval_clf.py 的第71行程式開始，要有對應的 CnonC。

In [ ]:
!python eval_clf.py CnonC ../Datasets/CnonC_test.txt Output_CnonC/test_results.tsv


Print out the F1-score of the result of BERT classifier for a data set.
  Usage:   python eval_clf.py Data_Name Test_File Predicted_File
  Example: python eval_clf.py CnonC CnonC_test.txt Output_CnonC/test_results.tsv

sys.argv: ['eval_clf.py', 'CnonC', '../Datasets/CnonC_test.txt', 'Output_CnonC/test_results.tsv']
002-非營建類    50
001-營建類     50
Name: 0, dtype: int64
From '../Datasets/CnonC_test.txt' file: df0
Label_List='['001-營建類', '002-非營建類']'
Number of Labels: 2
From copy of get_labels() in BERT's run_classifier.py
Label_List='['001-營建類', '002-非營建類']'
Number of Labels:2
Num of Classes (Categories or Labels): 2
<class 'numpy.ndarray'> Label Names [:2]: ['001-營建類' '002-非營建類']
Label Names transformed[:2]: [0 1]
Label inverse transform [0, 1]: ['001-營建類' '002-非營建類']


MicroF1 = 0.9100, MacroF1=0.9099
	Precision	Recall	F1	Support
Micro	0.91	0.91	0.91	None
Macro	0.9114	0.91	0.9099	None


[[47  3]
 [ 6 44]]

              precision    recall  f1-score   support

     001-營建類     0.8868   

##二、PCWeb資料集
###1. 訓練 PCWeb

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=pcweb --do_train=true --do_eval=false \
  --data_dir=../Datasets \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=chinese_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=256 \
  --train_batch_size=8 \
  --learning_rate=2e-5 \
  --num_train_epochs=20.0 \
  --output_dir=Model_PCWeb
print("It takes %4.2f seconds to train PCWeb."%(time.time()-time_Start))

###2. 預測 PCWeb

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=pcweb --do_predict=true \
  --data_dir=../Datasets \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=Model_PCWeb \
  --max_seq_length=256 \
  --output_dir=Output_PCWeb
print("It takes %4.2f seconds to predict PCWeb."%(time.time()-time_Start))

###3. 評估 PCWeb 成效

In [ ]:
!python eval_clf.py PCWeb ../Datasets/PCWeb_test.txt Output_PCWeb/test_results.tsv

##三、PCNews 資料集
###1. 訓練 PCNews

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=pcnews --do_train=true --do_eval=false \
  --data_dir=../Datasets \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=chinese_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=512 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=20.0 \
  --output_dir=Model_PCNews
print("It takes %4.2f seconds to train PCNews."%(time.time()-time_Start))

###2. 預測 PCNews

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=pcnews --do_predict=true \
  --data_dir=../Datasets \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=Model_PCNews \
  --max_seq_length=512 \
  --output_dir=Output_PCNews
print("It takes %4.2f seconds to predict PCNews."%(time.time()-time_Start))

###3. 評估 PCNews 成效

In [ ]:
!python eval_clf.py PCNews ../Datasets/PCNews_test.txt Output_PCNews/test_results.tsv

##四、Joke 資料集
###1. 訓練 Joke

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=joke --do_train=true --do_eval=false \
  --data_dir=../Datasets \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=chinese_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=512 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=20.0 \
  --output_dir=Model_Joke
print("It takes %4.2f seconds to train Joke."%(time.time()-time_Start))

###2. 預測 Joke

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=joke --do_predict=true \
  --data_dir=../Datasets \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=Model_Joke \
  --max_seq_length=512 \
  --output_dir=Output_Joke
print("It takes %4.2f seconds to predict Joke."%(time.time()-time_Start))

###3. 評估 Joke 成效

In [ ]:
!python eval_clf.py Joke ../Datasets/Joke_test.txt Output_Joke/test_results.tsv

##五、CTC 資料集
###1. 訓練 CTC

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=ctc --do_train=true --do_eval=false \
  --data_dir=../Datasets \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=chinese_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=512 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=20.0 \
  --output_dir=Model_CTC
print("It takes %4.2f seconds to train CTC."%(time.time()-time_Start))

###2. 預測 CTC

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=ctc --do_predict=true \
  --data_dir=../Datasets \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=Model_CTC \
  --max_seq_length=512 \
  --output_dir=Output_CTC
print("It takes %4.2f seconds to predict CTC."%(time.time()-time_Start))

###3. 評估 CTC 成效

In [ ]:
!python eval_clf.py CTC ../Datasets/CTC_test_sl.txt Output_CTC/test_results.tsv

##六、Reuters 資料集

###1. 訓練 Reuters
需要事先下載 BERT 的英文 model：uncased_L-12_H-768_A-12

In [ ]:
!rm -rf BERT/bert/Model_Reuters/*

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=reuters --do_train=true --do_eval=false \
  --data_dir=../Datasets \
  --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=256 \
  --train_batch_size=8 \
  --learning_rate=2e-5 \
  --num_train_epochs=20.0 \
  --output_dir=Model_Reuters
print("It takes %4.2f seconds to train Reuters."%(time.time()-time_Start))

###2. 測試 Reuters

In [ ]:
time_Start = time.time()
!python run_classifier_sam.py \
  --task_name=reuters --do_predict=true \
  --data_dir=../Datasets \
  --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=Model_Reuters \
  --max_seq_length=256 \
  --output_dir=Output_Reuters
print("It takes %4.2f seconds to predict Reuters."%(time.time()-time_Start))

###3. 評估 Reuters 成效

In [ ]:
!python eval_clf.py Reuters ../Datasets/Reuters_test_sl.txt Output_Reuters/test_results.tsv


Print out the F1-score of the result of BERT classifier for a data set.
  Usage:   python eval_clf.py Data_Name Test_File Predicted_File
  Example: python eval_clf.py HiNet HiNet_test.txt Output_Hinet/test_results.tsv

sys.argv: ['BERT/bert/eval_clf.py', 'Reuters', 'AnacondaProjects/Shared/TxtClf_Dataset/Reuters_test_sl.txt', 'BERT/bert/Output_Reuters/test_results.tsv']
The first 5 rows of 'AnacondaProjects/Shared/TxtClf_Dataset/Reuters_test_sl.txt':
The first 5 categories of 'AnacondaProjects/Shared/TxtClf_Dataset/Reuters_test_sl.txt'
0    interest
1    interest
2       trade
3        earn
4    money-fx
Name: 0, dtype: object
earn               1083
acq                 696
crude               121
money-fx             87
interest             81
trade                75
ship                 36
money-supply         28
sugar                25
coffee               22
gold                 20
alum                 19
cpi                  17
gnp                  15
cocoa                15
copp